In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, roc_auc_score

from scipy.sparse.csr import csr_matrix


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import gc
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
train = np.load('../input/multi-cat-encodings/X_train_ohe.npy', allow_pickle=True)
test = np.load('../input/multi-cat-encodings/X_test_ohe.npy', allow_pickle=True)
sample_submission = pd.read_csv('../input/cat-in-the-dat-ii/sample_submission.csv')
target = np.load('../input/multi-cat-encodings/target.npy')

In [ ]:
np.unique(target)

In [ ]:
sample_submission.head()

In [ ]:
train[()]

In [ ]:
train = train[()].tocsr()

In [ ]:
train_oof = np.zeros((600000,))
test_preds = 0
train_oof.shape

In [ ]:
test = test[()].tocsr()

In [ ]:
train[1]

In [ ]:
%%time
n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137)

for jj, (train_index, val_index) in enumerate(kf.split(train)):
    print("Fitting fold", jj+1)
    train_features = train[train_index]
    train_target = target[train_index]
    
    val_features = train[val_index]
    val_target = target[val_index]
    
    model = LogisticRegression(C=0.06)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred[:,1]
    print("Fold AUC:", roc_auc_score(val_target, val_pred[:,1]))
    test_preds += model.predict_proba(test)[:,1]/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

In [ ]:
print(roc_auc_score(target, train_oof))


In [ ]:
sample_submission['target'] = test_preds
sample_submission.to_csv('submission.csv', index=False)

In [ ]:
np.save('test_preds', test_preds)
np.save('train_oof', train_oof)